In [ ]:
# bootstrap_eval.py
import json
import time
import sys
from pathlib import Path
from typing import Dict, List, Tuple, Set
import numpy as np
import pandas as pd

# ----------------------------
# 1) CONFIG: your run folders
# ----------------------------
BASE_DIR = Path("")  # <- CHANGE THIS to the parent directory containing the run folders

models_labelset_paths = {
    "LAAT":       {"Original": "8i9s77f0", "EV": "kqo0vsqc", "ER": "5rj0tg1v"},
    "CAML":       {"Original": "p9hj5xe1", "EV": "05cvm0pf", "ER": "b1tl66nj"},
    "MultiResCNN":{"Original": "yz8bpcmd", "EV": "xxl6md4r", "ER": "974v2pcx"},
    "RNN":        {"Original": "9fd18hoi", "EV": "lrjwycyo", "ER": "rmxrgnnu"},
    "CNN":        {"Original": "o91zacs0", "EV": "qj24zl5u", "ER": "mlpgnhsf"},
    # add others here if needed
}

# Choose which label set to use as the EVALUATION reference for the test set:
EVAL_REFERENCE = "EV"        # options: "EV", "Original", "ER"
N_BOOT = 2000                      # bootstrap replicates
RNG = np.random.default_rng(13)

# How often to update the console progress line (in iterations)
PROGRESS_UPDATE_EVERY = 200

# Choose which label set to use as the EVALUATION reference for the test set:
EVAL_REFERENCE = "EV"  # options: "EV", "Original", "ER"

# Threshold tuning mode (for validation-based tuning)
#   "diagnostic" -> np.linspace(0.01, 0.99, 99)  (matches your diagnostic script)
#   "wide"       -> includes low thresholds (1e-6..0.5) for tiny scores
THRESHOLD_MODE = "diagnostic"   # or "wide"

if THRESHOLD_MODE == "diagnostic":
    THRESHOLD_GRID = np.linspace(0.01, 0.99, 99)
elif THRESHOLD_MODE == "wide":
    THRESHOLD_GRID = np.unique(np.concatenate([
        np.array([1e-6, 5e-6, 1e-5, 5e-5]),
        np.logspace(-4, -1, 13),   # 0.0001 → 0.1 (log-spaced)
        np.linspace(0.1, 0.5, 9),  # 0.1 → 0.5
    ]))
else:
    raise ValueError("THRESHOLD_MODE must be 'diagnostic' or 'wide'")

N_BOOT = 2000
RNG = np.random.default_rng(13)


# ----------------------------
# 2) Tiny progress helper
# ----------------------------
class Progress:
    def __init__(self, total: int, label: str = "Progress", update_every: int = PROGRESS_UPDATE_EVERY):
        self.total = int(total)
        self.label = label
        self.update_every = max(1, int(update_every))
        self.start = time.time()
        self.count = 0

    def update(self, n: int = 1):
        self.count += n
        if (self.count == 1) or (self.count % self.update_every == 0) or (self.count >= self.total):
            elapsed = time.time() - self.start
            rate = self.count / elapsed if elapsed > 0 else 0.0
            remaining = (self.total - self.count) / rate if rate > 0 else float("inf")
            pct = (self.count / self.total * 100.0) if self.total > 0 else 100.0
            msg = f"\r[{self.label}] {self.count}/{self.total} ({pct:.1f}%) | elapsed {elapsed:.1f}s | ETA {remaining:.1f}s"
            sys.stdout.write(msg)
            sys.stdout.flush()
            if self.count >= self.total:
                sys.stdout.write("\n")

# ----------------------------
# 3) Helpers
# ----------------------------
def ensure_list(x):
    import numpy as np
    import ast
    if x is None:
        return []
    if isinstance(x, (list, tuple)):
        return list(x)
    if isinstance(x, np.ndarray):
        return x.astype(str).tolist()
    if isinstance(x, str):
        try:
            val = ast.literal_eval(x)
            if isinstance(val, (list, tuple)):
                return [str(v) for v in val]
        except Exception:
            pass
    return []

def _read_feather(p: Path) -> pd.DataFrame:
    if not p.exists():
        raise FileNotFoundError(p)
    return pd.read_feather(p)

def _code_columns(df: pd.DataFrame) -> List[str]:
    # assumes non-code columns are exactly {'_id','target'}
    return [c for c in df.columns if c not in ("_id", "target")]

def _targets_dict(df: pd.DataFrame) -> Dict[str, Set[str]]:
    out = {}
    for _, row in df[["_id", "target"]].iterrows():
        nid = str(row["_id"])
        codes = set(row["target"]) if isinstance(row["target"], (list, tuple, set)) else set()
        out[nid] = codes
    return out

def _as_prob_matrix(df: pd.DataFrame, codes: List[str]) -> np.ndarray:
    """
    Return an (N x C) matrix of probabilities in [0,1].
    If values look like logits (outside [0,1]), apply sigmoid.
    """
    X = df[codes].to_numpy(dtype=float)
    xmin, xmax = np.nanmin(X), np.nanmax(X)
    if (xmin < -1e-6) or (xmax > 1 + 1e-6):
        X = 1.0 / (1.0 + np.exp(-X))  # sigmoid
    return X

def _pred_sets_from_df(df: pd.DataFrame, threshold: float) -> Dict[str, Set[str]]:
    codes = _code_columns(df)
    code_arr = np.array(codes, dtype=object)
    X = _as_prob_matrix(df, codes)
    meets = X >= threshold  # boolean mask
    id2pred = {}
    ids = df["_id"].astype(str).tolist()
    for i, nid in enumerate(ids):
        id2pred[nid] = set(code_arr[meets[i]])
    return id2pred

def _micro_counts(pred_sets: Dict[str, Set[str]],
                  true_sets: Dict[str, Set[str]],
                  ids: List[str]) -> Tuple[int,int,int]:
    TP = FP = FN = 0
    for nid in ids:
        p = pred_sets.get(nid, set())
        t = true_sets.get(nid, set())
        TP += len(p & t)
        FP += len(p - t)
        FN += len(t - p)
    return TP, FP, FN

def _micro_metrics_from_counts(TP:int, FP:int, FN:int) -> Tuple[float,float,float]:
    prec = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    rec  = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1   = (2 * prec * rec) / (prec + rec) if (prec + rec) > 0 else 0.0
    return prec, rec, f1

def _sweep_threshold_for_val(val_df: pd.DataFrame) -> float:
    """
    Pick a single global threshold that maximizes micro-F1 on the given val file.
    Grid is controlled by THRESHOLD_GRID, set from THRESHOLD_MODE at top.
    """
    codes = _code_columns(val_df)
    X = _as_prob_matrix(val_df, codes)
    true_sets = _targets_dict(val_df)
    ids = val_df["_id"].astype(str).tolist()

    best_t, best_f1 = float(THRESHOLD_GRID[0]), -1.0
    code_arr = np.array(codes, dtype=object)
    for t in THRESHOLD_GRID:
        meets = X >= t
        TP = FP = FN = 0
        for i, nid in enumerate(ids):
            p = set(code_arr[meets[i]])
            tset = true_sets.get(nid, set())
            TP += len(p & tset); FP += len(p - tset); FN += len(tset - p)
        _, _, f1 = _micro_metrics_from_counts(TP, FP, FN)
        if f1 > best_f1:
            best_f1, best_t = f1, float(t)
    return best_t


# NOTE: we’ll pass a callback into the bootstrap loops so we can update a single global progress bar.
def _bootstrap_ci_single(pred_sets: Dict[str, Set[str]],
                         true_sets: Dict[str, Set[str]],
                         ids: List[str],
                         B: int,
                         progress_cb=None) -> Dict[str, Tuple[float,float,float,float]]:
    """Returns dict(metric -> (point, lo, hi, std)) for precision, recall, microF1."""
    # point estimate on full sample
    TP, FP, FN = _micro_counts(pred_sets, true_sets, ids)
    p0, r0, f0 = _micro_metrics_from_counts(TP, FP, FN)

    # bootstrap
    n = len(ids)
    p_arr, r_arr, f_arr = np.empty(B), np.empty(B), np.empty(B)
    ids_np = np.array(ids, dtype=object)
    for b in range(B):
        idx = RNG.integers(0, n, size=n)  # sample notes with replacement
        boot_ids = ids_np[idx].tolist()
        TP, FP, FN = _micro_counts(pred_sets, true_sets, boot_ids)
        p, r, f = _micro_metrics_from_counts(TP, FP, FN)
        p_arr[b], r_arr[b], f_arr[b] = p, r, f
        if progress_cb:
            progress_cb(1)

    def _ci(a: np.ndarray):
        lo, hi = np.percentile(a, [2.5, 97.5])
        return lo, hi, float(a.std(ddof=1))
    p_lo, p_hi, p_sd = _ci(p_arr)
    r_lo, r_hi, r_sd = _ci(r_arr)
    f_lo, f_hi, f_sd = _ci(f_arr)
    return {
        "precision": (p0, p_lo, p_hi, p_sd),
        "recall":    (r0, r_lo, r_hi, r_sd),
        "f1_micro":  (f0, f_lo, f_hi, f_sd),
    }

def _paired_bootstrap_delta(predA: Dict[str, Set[str]],
                            predB: Dict[str, Set[str]],
                            true_sets: Dict[str, Set[str]],
                            ids: List[str],
                            B: int,
                            progress_cb=None) -> Dict[str, Tuple[float,float,float,float]]:
    """CI for delta metrics: A - B (paired over same bootstrapped note ids).
       Returns deltas and one-sided p-values for Precision, Recall, and F1 (test A > B)."""
    # point deltas (not used for p, but useful to keep consistent with CI)
    TP_A, FP_A, FN_A = _micro_counts(predA, true_sets, ids)
    TP_B, FP_B, FN_B = _micro_counts(predB, true_sets, ids)
    pA, rA, fA = _micro_metrics_from_counts(TP_A, FP_A, FN_A)
    pB, rB, fB = _micro_metrics_from_counts(TP_B, FP_B, FN_B)

    # paired bootstrap
    n = len(ids)
    ids_np = np.array(ids, dtype=object)
    dp, dr, df = np.empty(B), np.empty(B), np.empty(B)
    for b in range(B):
        idx = RNG.integers(0, n, size=n)
        boot_ids = ids_np[idx].tolist()
        TP_A, FP_A, FN_A = _micro_counts(predA, true_sets, boot_ids)
        TP_B, FP_B, FN_B = _micro_counts(predB, true_sets, boot_ids)
        pA, rA, fA = _micro_metrics_from_counts(TP_A, FP_A, FN_A)
        pB, rB, fB = _micro_metrics_from_counts(TP_B, FP_B, FN_B)
        dp[b], dr[b], df[b] = (pA - pB), (rA - rB), (fA - fB)
        if progress_cb:
            progress_cb(1)

    def _ci(a: np.ndarray):
        lo, hi = np.percentile(a, [2.5, 97.5])
        return float(a.mean()), float(lo), float(hi), float(a.std(ddof=1))

    # One-sided p-values for the hypothesis A > B
    p_one_sided_precision = float((dp <= 0).mean())
    p_one_sided_recall    = float((dr <= 0).mean())
    p_one_sided_f1        = float((df <= 0).mean())

    return {
        "d_precision": _ci(dp),
        "d_recall":    _ci(dr),
        "d_f1_micro":  _ci(df),
        "p_one_sided_precision": p_one_sided_precision,
        "p_one_sided_recall":    p_one_sided_recall,
        "p_one_sided_f1":        p_one_sided_f1,
    }


# ----------------------------
# 4) Main evaluation routine
# ----------------------------
def load_run(folder: Path):
    """Return (val_df, test_df) for a single run."""
    val_df  = _read_feather(folder / "predictions_val.feather")
    test_df = _read_feather(folder / "predictions_test.feather")

    # normalize targets to lists (handles numpy arrays or strings)
    val_df["target"]  = val_df["target"].apply(ensure_list)
    test_df["target"] = test_df["target"].apply(ensure_list)

    return val_df, test_df

def main():
    print(f"==> Stage 1/4: Loading runs from {BASE_DIR.resolve()}")
    per_arch = {}  # arch -> bundle
    total_runs = 0

    # Load + threshold tuning + prepare predictions/labels
    for arch, paths in models_labelset_paths.items():
        loaded = {}
        for labelset, folder_name in paths.items():
            folder = BASE_DIR / folder_name
            if not folder.exists():
                print(f"   [WARN] Missing folder: {folder}")
                continue
            total_runs += 1
            val_df, test_df = load_run(folder)

            # threshold tuned on this run's validation
            t0 = time.time()
            t = _sweep_threshold_for_val(val_df)
            print(f"   Loaded {arch}/{labelset} from {folder.name}: tuned threshold={t:.6f} (took {time.time()-t0:.1f}s)")

            # predicted sets for test at tuned threshold
            pred_sets = _pred_sets_from_df(test_df, t)
            # run's own ground-truth (used to pick reference later)
            true_sets = _targets_dict(test_df)
            ids = [str(x) for x in test_df["_id"].tolist()]

            loaded[labelset] = dict(
                folder=str(folder),
                threshold=t,
                pred_sets=pred_sets,
                true_sets=true_sets,
                ids=ids,
            )

        if not loaded:
            continue

        # reference labels inside this architecture:
        if EVAL_REFERENCE not in loaded:
            # fallback if requested reference missing
            fallback = next(iter(loaded.keys()))
            print(f"   [WARN] {arch}: requested EVAL_REFERENCE={EVAL_REFERENCE} not found. Falling back to {fallback}.")
            ref_true = loaded[fallback]["true_sets"]
        else:
            ref_true = loaded[EVAL_REFERENCE]["true_sets"]

        # intersect note ids across available labelsets for this arch
        common_ids = None
        for k in loaded:
            s = set(loaded[k]["ids"])
            common_ids = s if common_ids is None else (common_ids & s)
        common_ids = sorted(list(common_ids)) if common_ids else []

        per_arch[arch] = {
            "loaded": loaded,
            "ref_true_sets": ref_true,
            "common_ids": common_ids,
        }

    if not per_arch:
        print("No runs loaded. Check BASE_DIR and folder names.")
        return

    # Compute total bootstrap iterations for progress/ETA
    print("==> Stage 2/4: Planning bootstrap workload")
    total_boot_iters = 0
    for arch, bundle in per_arch.items():
        ids = bundle["common_ids"]
        if not ids:
            continue
        runs = bundle["loaded"]
        # per-model CIs (one per available labelset)
        total_boot_iters += len(runs) * N_BOOT
        # paired deltas (EV-Orig, ER-Orig) if present
        if "EV" in runs and "Original" in runs:
            total_boot_iters += N_BOOT
        if "ER" in runs and "Original" in runs:
            total_boot_iters += N_BOOT
    print(f"   Planned bootstrap iterations: {total_boot_iters}")

    progress = Progress(total=total_boot_iters, label="Bootstrapping", update_every=PROGRESS_UPDATE_EVERY)
    progress_cb = progress.update if total_boot_iters > 0 else None

    # 3a) Per-model (descriptive) CIs vs chosen reference labels
    print("==> Stage 3/4: Running per-model bootstrap CIs")
    per_model_records = []
    for arch, bundle in per_arch.items():
        ref_true = bundle["ref_true_sets"]
        ids = bundle["common_ids"]
        if not ids:
            print(f"   [INFO] {arch}: no common IDs across labelsets; skipping.")
            continue
        for labelset, run in bundle["loaded"].items():
            ci = _bootstrap_ci_single(run["pred_sets"], ref_true, ids, B=N_BOOT, progress_cb=progress_cb)
            per_model_records.append({
                "architecture": arch,
                "trained_on": labelset,
                "folder": run["folder"],
                "threshold": run["threshold"],
                "precision": ci["precision"][0],
                "precision_lo": ci["precision"][1],
                "precision_hi": ci["precision"][2],
                "recall": ci["recall"][0],
                "recall_lo": ci["recall"][1],
                "recall_hi": ci["recall"][2],
                "f1_micro": ci["f1_micro"][0],
                "f1_lo": ci["f1_micro"][1],
                "f1_hi": ci["f1_micro"][2],
            })
    per_model_df = pd.DataFrame(per_model_records).sort_values(["architecture", "trained_on"])

    # 3b) Paired bootstrap deltas inside each architecture (EV-Orig, ER-Orig)
    print("\n==> Stage 4/4: Running paired delta bootstraps")
    delta_records = []
    for arch, bundle in per_arch.items():
        ref_true = bundle["ref_true_sets"]
        ids = bundle["common_ids"]
        if not ids:
            continue
        runs = bundle["loaded"]
        # EV vs Original
        if "EV" in runs and "Original" in runs:
            d = _paired_bootstrap_delta(runs["EV"]["pred_sets"], runs["Original"]["pred_sets"], ref_true, ids, B=N_BOOT, progress_cb=progress_cb)
            delta_records.append({
                "architecture": arch, "comparison": "EV - Original",
                "d_precision": d["d_precision"][0], "d_precision_lo": d["d_precision"][1], "d_precision_hi": d["d_precision"][2],
                "d_recall":    d["d_recall"][0],    "d_recall_lo":    d["d_recall"][1],    "d_recall_hi":    d["d_recall"][2],
                "d_f1":        d["d_f1_micro"][0],  "d_f1_lo":        d["d_f1_micro"][1],  "d_f1_hi":        d["d_f1_micro"][2],
                "p_one_sided_precision": d["p_one_sided_precision"],
                "p_one_sided_recall":    d["p_one_sided_recall"],
                "p_one_sided_f1":        d["p_one_sided_f1"],
            })

        # ER vs Original
        if "ER" in runs and "Original" in runs:
            d = _paired_bootstrap_delta(runs["ER"]["pred_sets"], runs["Original"]["pred_sets"], ref_true, ids, B=N_BOOT, progress_cb=progress_cb)
            delta_records.append({
                "architecture": arch, "comparison": "ER - Original",
                "d_precision": d["d_precision"][0], "d_precision_lo": d["d_precision"][1], "d_precision_hi": d["d_precision"][2],
                "d_recall":    d["d_recall"][0],    "d_recall_lo":    d["d_recall"][1],    "d_recall_hi":    d["d_recall"][2],
                "d_f1":        d["d_f1_micro"][0],  "d_f1_lo":        d["d_f1_micro"][1],  "d_f1_hi":        d["d_f1_micro"][2],
                "p_one_sided_precision": d["p_one_sided_precision"],
                "p_one_sided_recall":    d["p_one_sided_recall"],
                "p_one_sided_f1":        d["p_one_sided_f1"],
            })
    delta_df = pd.DataFrame(delta_records).sort_values(["architecture", "comparison"])

    # Summary across architectures
    if not delta_df.empty:
        delta_df["sig_f1"] = (delta_df["d_f1_lo"] > 0).astype(int)
        by_comp = delta_df.groupby("comparison")["sig_f1"].agg(["sum", "count"]).reset_index()
        by_comp["proportion_sig"] = by_comp["sum"] / by_comp["count"]
    else:
        by_comp = pd.DataFrame(columns=["comparison", "sum", "count", "proportion_sig"])

    # Save results
    out_dir = BASE_DIR / "bootstrap_results"
    out_dir.mkdir(exist_ok=True)
    per_model_df.to_csv(out_dir / "per_model_ci.csv", index=False)
    delta_df.to_csv(out_dir / "paired_deltas_ci.csv", index=False)
    by_comp.to_csv(out_dir / "summary_across_architectures.csv", index=False)

    print("\n==> Done. Saved results to:")
    print(f"   {out_dir / 'per_model_ci.csv'}")
    print(f"   {out_dir / 'paired_deltas_ci.csv'}")
    print(f"   {out_dir / 'summary_across_architectures.csv'}")

if __name__ == "__main__":
    main()


==> Stage 1/4: Loading runs from /home/aabudaye/medical-coding-reproducibility/files
   Loaded LAAT/Original from 8i9s77f0: tuned threshold=0.320000 (took 1.5s)
   Loaded LAAT/EV from kqo0vsqc: tuned threshold=0.340000 (took 1.2s)
   Loaded LAAT/ER from 5rj0tg1v: tuned threshold=0.310000 (took 1.4s)
   Loaded CAML/Original from p9hj5xe1: tuned threshold=0.240000 (took 1.3s)
   Loaded CAML/EV from 05cvm0pf: tuned threshold=0.280000 (took 1.2s)
   Loaded CAML/ER from b1tl66nj: tuned threshold=0.280000 (took 1.4s)
   Loaded MultiResCNN/Original from yz8bpcmd: tuned threshold=0.220000 (took 1.3s)
   Loaded MultiResCNN/EV from xxl6md4r: tuned threshold=0.370000 (took 1.2s)
   Loaded MultiResCNN/ER from 974v2pcx: tuned threshold=0.330000 (took 1.4s)
   Loaded RNN/Original from 9fd18hoi: tuned threshold=0.170000 (took 1.2s)
   Loaded RNN/EV from lrjwycyo: tuned threshold=0.210000 (took 1.1s)
   Loaded RNN/ER from rmxrgnnu: tuned threshold=0.180000 (took 1.2s)
   Loaded CNN/Original from o91za

In [27]:
import pandas as pd
import matplotlib.pyplot as plt

per_model = pd.read_csv("bootstrap_results/per_model_ci.csv")
print(per_model.head())

# Quick summary table by architecture
summary = (
    per_model.groupby(["architecture","trained_on"])
    [["precision","recall","f1_micro"]].mean()
    .round(3)
)
summary


  architecture trained_on    folder  threshold  precision  precision_lo  \
0         CAML         ER  b1tl66nj       0.28   0.476531      0.463993   
1         CAML         EV  05cvm0pf       0.28   0.492162      0.479733   
2         CAML   Original  p9hj5xe1       0.24   0.388476      0.378122   
3          CNN         ER  mlpgnhsf       0.38   0.254274      0.189174   
4          CNN         EV  qj24zl5u       0.34   0.271605      0.200683   

   precision_hi    recall  recall_lo  recall_hi  f1_micro     f1_lo     f1_hi  
0      0.489653  0.369754   0.358417   0.380855  0.416406  0.406211  0.427178  
1      0.505628  0.393753   0.382122   0.405519  0.437492  0.426606  0.448394  
2      0.399540  0.428485   0.416919   0.440664  0.407501  0.397785  0.417816  
3      0.333080  0.200917   0.190920   0.210669  0.224468  0.194253  0.252738  
4      0.353367  0.228172   0.217953   0.237821  0.248001  0.213074  0.278574  


precision  recall  f1_micro
architecture trained_on                             
CAML         ER              0.477   0.370     0.416
             EV              0.492   0.394     0.437
             Original        0.388   0.428     0.408
CNN          ER              0.254   0.201     0.224
             EV              0.272   0.228     0.248
             Original        0.188   0.274     0.223
LAAT         ER              0.516   0.403     0.453
             EV              0.512   0.378     0.435
             Original        0.420   0.417     0.419
MultiResCNN  ER              0.536   0.310     0.393
             EV              0.529   0.362     0.429
             Original        0.367   0.371     0.369
RNN          ER              0.358   0.231     0.281
             EV              0.381   0.251     0.303
             Original        0.269   0.278     0.274

In [49]:
# bootstrap_self_eval.py
import json, time, sys
from pathlib import Path
from typing import Dict, List, Tuple, Set
import numpy as np
import pandas as pd

# ============================
# 1) CONFIG
# ============================
BASE_DIR = Path("")  # <- set to parent folder that contains run folders (e.g., "8i9s77f0", ...)
models_labelset_paths = {
    "LAAT":       {"Original": "8i9s77f0", "EV": "kqo0vsqc", "ER": "5rj0tg1v"},
    "CAML":       {"Original": "p9hj5xe1", "EV": "05cvm0pf", "ER": "b1tl66nj"},
    "MultiResCNN":{"Original": "yz8bpcmd", "EV": "xxl6md4r", "ER": "974v2pcx"},
    "RNN":        {"Original": "9fd18hoi", "EV": "lrjwycyo", "ER": "rmxrgnnu"},
    "CNN":        {"Original": "o91zacs0", "EV": "qj24zl5u", "ER": "mlpgnhsf"},
}

# Threshold tuning grid (match your diagnostic script or use the wide one if you ever need it)
THRESHOLD_MODE = "diagnostic"  # "diagnostic" -> np.linspace(0.01, 0.99, 99); "wide" -> includes very low thresholds
if THRESHOLD_MODE == "diagnostic":
    THRESHOLD_GRID = np.linspace(0.01, 0.99, 99)
elif THRESHOLD_MODE == "wide":
    THRESHOLD_GRID = np.unique(np.concatenate([
        np.array([1e-6, 5e-6, 1e-5, 5e-5]),
        np.logspace(-4, -1, 13),
        np.linspace(0.1, 0.5, 9),
    ]))
else:
    raise ValueError("THRESHOLD_MODE must be 'diagnostic' or 'wide'")

N_BOOT = 2000
RNG = np.random.default_rng(13)
PROGRESS_UPDATE_EVERY = 200

# ============================
# 2) Progress helper
# ============================
class Progress:
    def __init__(self, total: int, label: str = "Progress", update_every: int = PROGRESS_UPDATE_EVERY):
        self.total = int(total)
        self.label = label
        self.update_every = max(1, int(update_every))
        self.start = time.time()
        self.count = 0
    def update(self, n: int = 1):
        self.count += n
        if (self.count == 1) or (self.count % self.update_every == 0) or (self.count >= self.total):
            elapsed = time.time() - self.start
            rate = self.count / elapsed if elapsed > 0 else 0.0
            remaining = (self.total - self.count) / rate if rate > 0 else float("inf")
            pct = (self.count / self.total * 100.0) if self.total > 0 else 100.0
            msg = f"\r[{self.label}] {self.count}/{self.total} ({pct:.1f}%) | elapsed {elapsed:.1f}s | ETA {remaining:.1f}s"
            sys.stdout.write(msg); sys.stdout.flush()
            if self.count >= self.total: sys.stdout.write("\n")

# ============================
# 3) Helpers
# ============================
def ensure_list(x):
    import numpy as np, ast
    if x is None: return []
    if isinstance(x, (list, tuple)): return list(x)
    if isinstance(x, np.ndarray): return x.astype(str).tolist()
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            if isinstance(v, (list, tuple)): return [str(i) for i in v]
        except Exception: pass
    return []

def _read_feather(p: Path) -> pd.DataFrame:
    if not p.exists(): raise FileNotFoundError(p)
    return pd.read_feather(p)

def _code_columns(df: pd.DataFrame) -> List[str]:
    return [c for c in df.columns if c not in ("_id", "target")]

def _targets_dict(df: pd.DataFrame) -> Dict[str, Set[str]]:
    out = {}
    for _, row in df[["_id","target"]].iterrows():
        out[str(row["_id"])] = set(row["target"]) if isinstance(row["target"], (list, tuple, set)) else set()
    return out

def _as_prob_matrix(df: pd.DataFrame, codes: List[str]) -> np.ndarray:
    X = df[codes].to_numpy(dtype=float)
    xmin, xmax = np.nanmin(X), np.nanmax(X)
    if (xmin < -1e-6) or (xmax > 1 + 1e-6):
        X = 1.0 / (1.0 + np.exp(-X))  # sigmoid only if outside [0,1]
    return X

def _pred_sets_from_df(df: pd.DataFrame, threshold: float) -> Dict[str, Set[str]]:
    codes = _code_columns(df)
    code_arr = np.array(codes, dtype=object)
    X = _as_prob_matrix(df, codes)
    meets = X >= threshold
    ids = df["_id"].astype(str).tolist()
    return {nid: set(code_arr[meets[i]]) for i, nid in enumerate(ids)}

def _micro_counts(pred_sets: Dict[str, Set[str]],
                  true_sets: Dict[str, Set[str]],
                  ids: List[str]) -> Tuple[int,int,int]:
    TP=FP=FN=0
    for nid in ids:
        p = pred_sets.get(nid, set()); t = true_sets.get(nid, set())
        TP += len(p & t); FP += len(p - t); FN += len(t - p)
    return TP, FP, FN

def _micro_metrics_from_counts(TP:int, FP:int, FN:int) -> Tuple[float,float,float]:
    prec = TP/(TP+FP) if (TP+FP)>0 else 0.0
    rec  = TP/(TP+FN) if (TP+FN)>0 else 0.0
    f1   = (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0
    return prec, rec, f1

def _sweep_threshold_for_val(val_df: pd.DataFrame) -> float:
    codes = _code_columns(val_df)
    X = _as_prob_matrix(val_df, codes)
    true_sets = _targets_dict(val_df)
    ids = val_df["_id"].astype(str).tolist()
    best_t, best_f1 = float(THRESHOLD_GRID[0]), -1.0
    code_arr = np.array(codes, dtype=object)
    for t in THRESHOLD_GRID:
        meets = X >= t
        TP=FP=FN=0
        for i, nid in enumerate(ids):
            p = set(code_arr[meets[i]]); tset = true_sets.get(nid, set())
            TP += len(p & tset); FP += len(p - tset); FN += len(tset - p)
        _,_,f1 = _micro_metrics_from_counts(TP,FP,FN)
        if f1 > best_f1: best_f1, best_t = f1, float(t)
    return best_t

def _bootstrap_ci_single(pred_sets: Dict[str, Set[str]],
                         true_sets: Dict[str, Set[str]],
                         ids: List[str],
                         B: int,
                         progress_cb=None) -> Dict[str, Tuple[float,float,float,float]]:
    TP,FP,FN = _micro_counts(pred_sets, true_sets, ids)
    p0,r0,f0 = _micro_metrics_from_counts(TP,FP,FN)
    n = len(ids); ids_np = np.array(ids, dtype=object)
    p_arr = np.empty(B); r_arr = np.empty(B); f_arr = np.empty(B)
    for b in range(B):
        idx = RNG.integers(0, n, size=n)
        boot_ids = ids_np[idx].tolist()
        TP,FP,FN = _micro_counts(pred_sets, true_sets, boot_ids)
        p,r,f = _micro_metrics_from_counts(TP,FP,FN)
        p_arr[b], r_arr[b], f_arr[b] = p,r,f
        if progress_cb: progress_cb(1)
    def _ci(a):
        lo, hi = np.percentile(a, [2.5,97.5]); return lo, hi, float(a.std(ddof=1))
    p_lo,p_hi,p_sd = _ci(p_arr); r_lo,r_hi,r_sd = _ci(r_arr); f_lo,f_hi,f_sd = _ci(f_arr)
    return {"precision":(p0,p_lo,p_hi,p_sd), "recall":(r0,r_lo,r_hi,r_sd), "f1_micro":(f0,f_lo,f_hi,f_sd)}

def _paired_bootstrap_delta_selfref(predA: Dict[str, Set[str]], trueA: Dict[str, Set[str]],
                                    predB: Dict[str, Set[str]], trueB: Dict[str, Set[str]],
                                    ids: List[str], B:int, progress_cb=None):
    """Paired bootstrap over SAME note IDs; A and B each use their own truths.
       Returns deltas and one-sided p-values for Precision, Recall, and F1 (test A > B)."""
    # point
    TP_A,FP_A,FN_A = _micro_counts(predA, trueA, ids)
    TP_B,FP_B,FN_B = _micro_counts(predB, trueB, ids)
    pA,rA,fA = _micro_metrics_from_counts(TP_A,FP_A,FN_A)
    pB,rB,fB = _micro_metrics_from_counts(TP_B,FP_B,FN_B)

    n = len(ids); ids_np = np.array(ids, dtype=object)
    dp = np.empty(B); dr = np.empty(B); df = np.empty(B)
    for b in range(B):
        idx = RNG.integers(0, n, size=n)
        boot_ids = ids_np[idx].tolist()
        TP_A,FP_A,FN_A = _micro_counts(predA, trueA, boot_ids)
        TP_B,FP_B,FN_B = _micro_counts(predB, trueB, boot_ids)
        pA,rA,fA = _micro_metrics_from_counts(TP_A,FP_A,FN_A)
        pB,rB,fB = _micro_metrics_from_counts(TP_B,FP_B,FN_B)
        dp[b], dr[b], df[b] = (pA-pB), (rA-rB), (fA-fB)
        if progress_cb: progress_cb(1)

    def _ci(a):
        lo, hi = np.percentile(a, [2.5,97.5]); return float(a.mean()), float(lo), float(hi), float(a.std(ddof=1))

    # One-sided p-values for "A > B" (i.e., delta <= 0 is evidence against)
    p_one_sided_precision = float((dp <= 0).mean())
    p_one_sided_recall    = float((dr <= 0).mean())
    p_one_sided_f1        = float((df <= 0).mean())

    return {
        "d_precision": _ci(dp),
        "d_recall":    _ci(dr),
        "d_f1_micro":  _ci(df),
        "p_one_sided_precision": p_one_sided_precision,
        "p_one_sided_recall":    p_one_sided_recall,
        "p_one_sided_f1":        p_one_sided_f1,
    }

# ============================
# 4) Main
# ============================
def load_run(folder: Path):
    val_df  = _read_feather(folder/"predictions_val.feather")
    test_df = _read_feather(folder/"predictions_test.feather")
    val_df["target"]  = val_df["target"].apply(ensure_list)
    test_df["target"] = test_df["target"].apply(ensure_list)
    return val_df, test_df

def main():
    print(f"==> Stage 1/4: Loading & tuning (self-referenced)")
    per_arch = {}
    total_runs = 0
    for arch, paths in models_labelset_paths.items():
        loaded = {}
        for labelset, folder_name in paths.items():
            folder = BASE_DIR / folder_name
            if not folder.exists():
                print(f"   [WARN] Missing folder: {folder}")
                continue
            total_runs += 1
            val_df, test_df = load_run(folder)
            t0 = time.time()
            thr = _sweep_threshold_for_val(val_df)  # tuned on this run's own val targets
            print(f"   Loaded {arch}/{labelset} from {folder.name}: tuned threshold={thr:.6f} (took {time.time()-t0:.1f}s)")
            pred_sets = _pred_sets_from_df(test_df, thr)  # test predictions binarized
            true_sets = _targets_dict(test_df)            # this run's OWN test targets
            ids = [str(x) for x in test_df["_id"].tolist()]
            loaded[labelset] = dict(folder=str(folder), threshold=thr, pred_sets=pred_sets, true_sets=true_sets, ids=ids)
        if not loaded: continue
        # common note ids across available labelsets (so deltas use same notes)
        common_ids = None
        for k in loaded:
            s = set(loaded[k]["ids"])
            common_ids = s if common_ids is None else (common_ids & s)
        per_arch[arch] = {"loaded": loaded, "common_ids": sorted(list(common_ids)) if common_ids else []}

    if not per_arch:
        print("No runs loaded. Check BASE_DIR and folder names.")
        return

    # plan workload
    print("==> Stage 2/4: Planning bootstrap workload")
    total_boot_iters = 0
    for arch, bundle in per_arch.items():
        ids = bundle["common_ids"]; runs = bundle["loaded"]
        if not ids: continue
        total_boot_iters += len(runs) * N_BOOT  # per-model CIs
        # paired deltas available whenever both sides exist
        if "EV" in runs and "Original" in runs: total_boot_iters += N_BOOT
        if "ER" in runs and "Original" in runs: total_boot_iters += N_BOOT
        if "EV" in runs and "ER" in runs:       total_boot_iters += N_BOOT
    print(f"   Planned bootstrap iterations: {total_boot_iters}")
    progress = Progress(total=total_boot_iters, label="Bootstrapping", update_every=PROGRESS_UPDATE_EVERY)
    progress_cb = progress.update if total_boot_iters > 0 else None

    # 3a) per-model self-referenced CIs
    print("==> Stage 3/4: Per-model self-referenced CIs")
    per_model_records = []
    for arch, bundle in per_arch.items():
        ids = bundle["common_ids"]; runs = bundle["loaded"]
        if not ids:
            print(f"   [INFO] {arch}: no common IDs; skipping.")
            continue
        for labelset, run in runs.items():
            ci = _bootstrap_ci_single(run["pred_sets"], run["true_sets"], ids, B=N_BOOT, progress_cb=progress_cb)
            per_model_records.append({
                "architecture": arch,
                "trained_on": labelset,
                "folder": run["folder"],
                "threshold": run["threshold"],
                "precision": ci["precision"][0], "precision_lo": ci["precision"][1], "precision_hi": ci["precision"][2],
                "recall":    ci["recall"][0],    "recall_lo":    ci["recall"][1],    "recall_hi":    ci["recall"][2],
                "f1_micro":  ci["f1_micro"][0],  "f1_lo":        ci["f1_micro"][1],  "f1_hi":        ci["f1_micro"][2],
            })
    per_model_df = pd.DataFrame(per_model_records).sort_values(["architecture","trained_on"])

    # 3b) paired deltas (still self-referenced per side), resampling the same notes
    print("\n==> Stage 4/4: Paired deltas (self-referenced on each side)")
    delta_records = []
    for arch, bundle in per_arch.items():
        ids = bundle["common_ids"]; runs = bundle["loaded"]
        if not ids: continue
        def add_delta(nameA, nameB, label):
            d = _paired_bootstrap_delta_selfref(
                runs[nameA]["pred_sets"], runs[nameA]["true_sets"],
                runs[nameB]["pred_sets"], runs[nameB]["true_sets"],
                ids, B=N_BOOT, progress_cb=progress_cb)
            delta_records.append({
                "architecture": arch, "comparison": label,
                "d_precision": d["d_precision"][0], "d_precision_lo": d["d_precision"][1], "d_precision_hi": d["d_precision"][2],
                "d_recall":    d["d_recall"][0],    "d_recall_lo":    d["d_recall"][1],    "d_recall_hi":    d["d_recall"][2],
                "d_f1":        d["d_f1_micro"][0],  "d_f1_lo":        d["d_f1_micro"][1],  "d_f1_hi":        d["d_f1_micro"][2],
                "p_one_sided_precision": d["p_one_sided_precision"],
                "p_one_sided_recall":    d["p_one_sided_recall"],
                "p_one_sided_f1":        d["p_one_sided_f1"],
            })
        if "EV" in runs and "Original" in runs: add_delta("EV","Original","EV - Original (self-ref)")
        if "ER" in runs and "Original" in runs: add_delta("ER","Original","ER - Original (self-ref)")
        if "EV" in runs and "ER" in runs:       add_delta("EV","ER","EV - ER (self-ref)")
    delta_df = pd.DataFrame(delta_records).sort_values(["architecture","comparison"])

    # save
    out_dir = BASE_DIR / "bootstrap_results_selfref"
    out_dir.mkdir(exist_ok=True)
    per_model_df.to_csv(out_dir/"per_model_ci.csv", index=False)
    delta_df.to_csv(out_dir/"paired_deltas_ci.csv", index=False)

    print("\n==> Done. Saved results to:")
    print(f"   {out_dir/'per_model_ci.csv'}")
    print(f"   {out_dir/'paired_deltas_ci.csv'}")

if __name__ == "__main__":
    main()


==> Stage 1/4: Loading & tuning (self-referenced)
   Loaded LAAT/Original from 8i9s77f0: tuned threshold=0.320000 (took 1.5s)
   Loaded LAAT/EV from kqo0vsqc: tuned threshold=0.340000 (took 1.3s)
   Loaded LAAT/ER from 5rj0tg1v: tuned threshold=0.310000 (took 1.5s)
   Loaded CAML/Original from p9hj5xe1: tuned threshold=0.240000 (took 1.4s)
   Loaded CAML/EV from 05cvm0pf: tuned threshold=0.280000 (took 1.2s)
   Loaded CAML/ER from b1tl66nj: tuned threshold=0.280000 (took 1.4s)
   Loaded MultiResCNN/Original from yz8bpcmd: tuned threshold=0.220000 (took 1.3s)
   Loaded MultiResCNN/EV from xxl6md4r: tuned threshold=0.370000 (took 1.2s)
   Loaded MultiResCNN/ER from 974v2pcx: tuned threshold=0.330000 (took 1.3s)
   Loaded RNN/Original from 9fd18hoi: tuned threshold=0.170000 (took 1.3s)
   Loaded RNN/EV from lrjwycyo: tuned threshold=0.210000 (took 1.2s)
   Loaded RNN/ER from rmxrgnnu: tuned threshold=0.180000 (took 1.3s)
   Loaded CNN/Original from o91zacs0: tuned threshold=0.280000 (took

In [20]:
import pandas as pd
import matplotlib.pyplot as plt

per_model = pd.read_csv("bootstrap_results_selfref/per_model_ci.csv")
print(per_model.head())

# Quick summary table by architecture
summary = (
    per_model.groupby(["architecture","trained_on"])
    [["precision","recall","f1_micro"]].mean()
    .round(3)
)
summary


  architecture trained_on    folder  threshold  precision  precision_lo  \
0         CAML         ER  b1tl66nj       0.28   0.466273      0.453448   
1         CAML         EV  05cvm0pf       0.28   0.492162      0.479733   
2         CAML   Original  p9hj5xe1       0.24   0.440739      0.429978   
3          CNN         ER  mlpgnhsf       0.38   0.248779      0.185121   
4          CNN         EV  qj24zl5u       0.34   0.271605      0.200683   

   precision_hi    recall  recall_lo  recall_hi  f1_micro     f1_lo     f1_hi  
0      0.479666  0.365097   0.353427   0.376659  0.409528  0.398828  0.420742  
1      0.505628  0.393753   0.382122   0.405519  0.437492  0.426606  0.448394  
2      0.451846  0.364269   0.353817   0.374460  0.398872  0.389028  0.408493  
3      0.326971  0.198369   0.188234   0.208049  0.220733  0.190648  0.248949  
4      0.353367  0.228172   0.217953   0.237821  0.248001  0.213074  0.278574  


precision  recall  f1_micro
architecture trained_on                             
CAML         ER              0.466   0.365     0.410
             EV              0.492   0.394     0.437
             Original        0.441   0.364     0.399
CNN          ER              0.249   0.198     0.221
             EV              0.272   0.228     0.248
             Original        0.224   0.244     0.234
LAAT         ER              0.501   0.395     0.442
             EV              0.512   0.378     0.435
             Original        0.478   0.356     0.408
MultiResCNN  ER              0.521   0.304     0.384
             EV              0.529   0.362     0.429
             Original        0.424   0.321     0.366
RNN          ER              0.348   0.227     0.275
             EV              0.381   0.251     0.303
             Original        0.334   0.259     0.292

In [37]:
# Plot ΔPrecision / ΔF1 / ΔRecall with 95% CIs + p-value stars (Precision/F1/Recall)
# Groups: EV−OM (top) then ER−OM (bottom), each with a single rotated label + square bracket
# Saves: figures/fixed_reference_all_metrics.png
#        figures/self_reference_all_metrics.png
#        figures/combined_side_by_side.png

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms

# ---------- config ----------
fixed_dir = "bootstrap_results"
self_dir  = "bootstrap_results_selfref"
out_dir   = "figures"

ARCH_ORDER = ["LAAT", "CAML", "MultiResCNN", "RNN", "CNN"]  # enforced within each block
COMP_ORDER = ["EV−OM", "ER−OM"]                              # block order top→bottom

# Color/marker settings: keep colored, but ensure B/W differentiability via darker hues + black edges
COLOR_MAP  = {
    "Precision": "#377eb8",  # dark-ish blue
    "F1":        "#e41a1c",  # dark-ish red
    "Recall":    "#4daf4a",  # dark-ish green
}
MARKER_MAP = {"Precision": "s", "F1": "o", "Recall": "D"}
Y_OFFSETS  = {"Precision": +0.25, "F1": 0.00, "Recall": -0.25}  # Precision up, F1 mid, Recall down

# Legend controls (tweak these if you want different sizes/layout later)
LEGEND_MARKERSCALE = 1.3
LEGEND_FONT_SIZE   = 6
LEGEND_NCOL        = 3

Path(out_dir).mkdir(parents=True, exist_ok=True)

def p_to_stars(p):
    try:
        if np.isnan(p): return ""
    except Exception:
        return ""
    if p <= 0.001: return "***"
    if p <= 0.01:  return "**"
    if p <= 0.05:  return "*"
    return ""

def _norm_comp_label(raw):
    """Map both fixed and self-ref CSV comparison strings to EV−OM / ER−OM (no '(self)')."""
    raw = str(raw)
    if raw.startswith("EV - Original"): return "EV−OM"
    if raw.startswith("ER - Original"): return "ER−OM"
    return raw

def load_paired_deltas_csv(csv_path):
    """Load paired_deltas_ci.csv and reshape to long format with p-values per metric."""
    df = pd.read_csv(csv_path)

    # Keep only EV-OM and ER-OM comparisons (both fixed and self files may contain self-ref suffix)
    df = df[df["comparison"].str.startswith(("EV - Original", "ER - Original"))].copy()
    df["Comparison"] = df["comparison"].map(_norm_comp_label)

    rows = []
    for _, r in df.iterrows():
        arch = r["architecture"]
        comp = r["Comparison"]
        rows.append(dict(Architecture=arch, Comparison=comp, Metric="Precision",
                         Delta=r["d_precision"], Lo=r["d_precision_lo"], Hi=r["d_precision_hi"],
                         p=r.get("p_one_sided_precision", np.nan)))
        rows.append(dict(Architecture=arch, Comparison=comp, Metric="F1",
                         Delta=r["d_f1"], Lo=r["d_f1_lo"], Hi=r["d_f1_hi"],
                         p=r.get("p_one_sided_f1", np.nan)))
        rows.append(dict(Architecture=arch, Comparison=comp, Metric="Recall",
                         Delta=r["d_recall"], Lo=r["d_recall_lo"], Hi=r["d_recall_hi"],
                         p=r.get("p_one_sided_recall", np.nan)))

    out = pd.DataFrame(rows)
    out["Comparison"]   = pd.Categorical(out["Comparison"],   categories=COMP_ORDER, ordered=True)
    out["Architecture"] = pd.Categorical(out["Architecture"], categories=ARCH_ORDER, ordered=True)
    out = out.sort_values(["Comparison", "Architecture", "Metric"]).reset_index(drop=True)
    return out

def plot_forest_on_ax(ax, df_long, title=None, xlim=None, add_legend=False):
    """Draw one panel with:
       - EV−OM block (top) then ER−OM (bottom)
       - single rotated label + square bracket per block
       - p-value stars for Precision, F1, Recall (smaller & closer)
       - thick black separator between blocks
       - y-tick labels = model names only, vertical
    """
    df = df_long.copy()

    # Build row order = for each comp (EV−OM then ER−OM), for each arch in fixed order
    unique_rows = (
        df.drop_duplicates(subset=["Comparison","Architecture"])[["Comparison","Architecture"]]
          .sort_values(["Comparison","Architecture"])
          .reset_index(drop=True)
    )
    unique_rows["RowKey"] = unique_rows["Comparison"].astype(str) + " · " + unique_rows["Architecture"].astype(str)
    row_keys = unique_rows["RowKey"].tolist()

    # y-base: reverse so first is top
    y_base_map = {rk: i for i, rk in enumerate(row_keys[::-1])}

    # Attach y_base to each metric row
    df["RowKey"] = df["Comparison"].astype(str) + " · " + df["Architecture"].astype(str)
    df["y_base"] = df["RowKey"].map(y_base_map)

    # Plot metrics with offsets and p-value stars
    for metric in ["Precision", "F1", "Recall"]:
        sub = df[df["Metric"] == metric]
        if sub.empty:
            continue
        y = sub["y_base"].values + Y_OFFSETS[metric]
        x = sub["Delta"].values
        xerr = np.vstack([x - sub["Lo"].values, sub["Hi"].values - x])
        ax.errorbar(
            x, y, xerr=xerr,
            fmt=MARKER_MAP[metric],
            markersize=7,               # bigger symbols
            mfc=COLOR_MAP[metric],      # colored face
            mec="black", mew=0.7,       # black edge for B/W clarity
            capsize=0,
            linestyle="None", label=metric,
            color=COLOR_MAP[metric], ecolor="black",  # black CI edges for B/W
            elinewidth=1.0, zorder=3
        )
        # p-value stars
        for xi, yi, pi in zip(x, y, sub["p"].values):
            stars = p_to_stars(pi)
            if stars:
                ax.text(
                    xi, yi + 0.03,
                    stars,
                    fontsize=7,
                    ha="center", va="bottom",
                    color="black", zorder=4
                )

    # Symmetric y-limits so top/bottom gap = inter-row gap
    if df["y_base"].empty:
        return ax
    ybases = sorted(df["y_base"].unique())
    ax.set_ylim(ybases[0] - 0.5, ybases[-1] + 0.5)

    # x-limits & separators
    if xlim is not None:
        ax.set_xlim(xlim)
    x0, x1 = ax.get_xlim()
    for yb in ybases[:-1]:
        ax.hlines(yb + 0.5, x0, x1, colors="#dddddd", linestyles="-", linewidth=0.8, zorder=1)

    # Vertical reference at 0
    ax.axvline(0, linestyle="--", color="gray", linewidth=1, zorder=1)

    # y ticks & labels — model names ONLY, vertical
    display_rows = unique_rows.iloc[::-1].reset_index(drop=True)  # top→bottom display
    display_rows["RowKey"] = display_rows["Comparison"].astype(str) + " · " + display_rows["Architecture"].astype(str)
    yticks  = [y_base_map[rk] for rk in row_keys[::-1]]
    ylabels = display_rows["Architecture"].tolist()  # only model name
    ax.set_yticks(yticks)
    ax.set_yticklabels(ylabels, rotation=90, va="center", ha="center", fontsize=6.5)  # vertical, centered

    # Thick black separator between EV−OM and ER−OM blocks
    display_comp = display_rows["Comparison"].tolist()
    display_base = [y_base_map[rk] for rk in display_rows["RowKey"]]
    for i in range(len(display_rows) - 1):
        if display_comp[i] != display_comp[i + 1]:
            y_boundary = display_base[i] + 0.5
            ax.hlines(y_boundary, x0, x1, colors="black", linewidth=1.8, zorder=2)

    # Single rotated bracketed block label in LEFT MARGIN for each comparison
    trans = mtransforms.blended_transform_factory(ax.transAxes, ax.transData)
    x_bracket = -0.055
    x_text    = -0.075
    bracket_tick = 0.015

    i = 0
    total = len(display_rows)
    while i < total:
        comp_i = display_comp[i]
        j = i
        while j < total and display_comp[j] == comp_i:
            j += 1
        bases = [display_base[k] for k in range(i, j)]
        if bases:
            y_top    = max(bases) + 0.25
            y_bottom = min(bases) - 0.25
            y_mid    = 0.5 * (y_top + y_bottom)

            ax.plot([x_bracket, x_bracket], [y_bottom, y_top], color="black", lw=1.2,
                    transform=trans, clip_on=False, zorder=2)
            ax.plot([x_bracket, x_bracket + bracket_tick], [y_top, y_top], color="black",
                    lw=1.2, transform=trans, clip_on=False, zorder=2)
            ax.plot([x_bracket, x_bracket + bracket_tick], [y_bottom, y_bottom], color="black",
                    lw=1.2, transform=trans, clip_on=False, zorder=2)

            ax.text(x_text, y_mid, comp_i, rotation=90, ha="center", va="center",
                    fontsize=10, color="black", weight="bold",
                    transform=trans, clip_on=False, zorder=5)
        i = j

    ax.set_xlabel("Δ (95% CI - Confidence Interval)", fontsize=7.5)
    if title:
        ax.set_title(title, pad=8)

    if add_legend:
        leg = ax.legend(
            loc="lower right",
            bbox_to_anchor=(0.2, -0.23),
            ncol=LEGEND_NCOL,
            frameon=False,
            markerscale=LEGEND_MARKERSCALE,
        )
        for t in leg.get_texts():
            t.set_fontsize(LEGEND_FONT_SIZE)

    # Wider left margin for bracket + label; small bottom for legend
    ax.figure.subplots_adjust(left=0.25, bottom=0.16)
    return ax

# ----- load data -----
fixed_long = load_paired_deltas_csv(os.path.join(fixed_dir, "paired_deltas_ci.csv"))
self_long  = load_paired_deltas_csv(os.path.join(self_dir,  "paired_deltas_ci.csv"))

# unified x-range for all panels
xmin = min(fixed_long["Lo"].min(), self_long["Lo"].min())
xmax = max(fixed_long["Hi"].max(), self_long["Hi"].max())
xlim = (float(xmin - 0.01), float(xmax + 0.01))

# Convenience: subsets per comparison
fixed_ev = fixed_long[fixed_long["Comparison"] == "EV−OM"]
fixed_er = fixed_long[fixed_long["Comparison"] == "ER−OM"]
self_ev  = self_long[self_long["Comparison"]  == "EV−OM"]
self_er  = self_long[self_long["Comparison"]  == "ER−OM"]

# ----- fixed-reference (now side-by-side EV−OM | ER−OM) -----
fig1, axes1 = plt.subplots(1, 2, figsize=(14, 3.6), sharex=True)
plot_forest_on_ax(axes1[0], fixed_ev, title="", xlim=xlim, add_legend=False)
plot_forest_on_ax(axes1[1], fixed_er, title="", xlim=xlim, add_legend=True)
fig1.suptitle("Fixed-reference: ΔPrecision / ΔF1 / ΔRecall", y=0.98)
fig1.subplots_adjust(wspace=0.25, bottom=0.16)
fig1.savefig(os.path.join(out_dir, "fixed_reference_all_metrics.png"), dpi=300, bbox_inches="tight", pad_inches=0.08)
plt.close(fig1)

# ----- self-referenced (now side-by-side EV−OM | ER−OM) -----
fig2, axes2 = plt.subplots(1, 2, figsize=(14, 3.6), sharex=True)
plot_forest_on_ax(axes2[0], self_ev,  title="", xlim=xlim, add_legend=False)
plot_forest_on_ax(axes2[1], self_er,  title="", xlim=xlim, add_legend=True)
fig2.suptitle("", y=0.98)
fig2.subplots_adjust(wspace=0.25, bottom=0.16)
fig2.savefig(os.path.join(out_dir, "self_reference_all_metrics.png"), dpi=300, bbox_inches="tight", pad_inches=0.08)
plt.close(fig2)

# ----- combined now STACKED vertically (top row = fixed side-by-side, bottom row = self side-by-side) -----
fig = plt.figure(figsize=(14, 13.6))
gs = fig.add_gridspec(nrows=2, ncols=2, hspace=0.35, wspace=0.25)

ax_tl = fig.add_subplot(gs[0, 0])
ax_tr = fig.add_subplot(gs[0, 1])
ax_bl = fig.add_subplot(gs[1, 0])
ax_br = fig.add_subplot(gs[1, 1])

# top row: fixed
plot_forest_on_ax(ax_tl, fixed_ev, title="(A) Fixed-reference — EV−OM", xlim=xlim, add_legend=False)
plot_forest_on_ax(ax_tr, fixed_er, title="(B) Fixed-reference — ER−OM", xlim=xlim, add_legend=True)

# bottom row: self
plot_forest_on_ax(ax_bl, self_ev,  title="(C) Self-referenced — EV−OM", xlim=xlim, add_legend=False)
plot_forest_on_ax(ax_br, self_er,  title="(D) Self-referenced — ER−OM", xlim=xlim, add_legend=True)

fig.subplots_adjust(bottom=0.16)
fig.savefig(os.path.join(out_dir, "combined_side_by_side.png"), dpi=300, bbox_inches="tight", pad_inches=0.08)
plt.close(fig)

print("Saved:",
      os.path.join(out_dir, "fixed_reference_all_metrics.png"),
      os.path.join(out_dir, "self_reference_all_metrics.png"),
      os.path.join(out_dir, "combined_side_by_side.png"))


Saved: figures\fixed_reference_all_metrics.png figures\self_reference_all_metrics.png figures\combined_side_by_side.png


In [48]:
# Per-model forest plots with OM→EV→ER order and thick black separators between architectures
# Inputs:
#   bootstrap_results/per_model_ci.csv
#   bootstrap_results_selfref/per_model_ci.csv
# Outputs:
#   figures/per_model_fixed_reference.png
#   figures/per_model_self_reference.png
#   figures/per_model_combined_side_by_side.png

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- config ----------
fixed_dir = "bootstrap_results"
self_dir  = "bootstrap_results_selfref"
out_dir   = "figures"

ARCH_ORDER  = ["LAAT", "CAML", "MultiResCNN", "RNN", "CNN"]   # enforce this always
TRAIN_ORDER = ["OM", "EV", "ER"]  # row order within each architecture

# light colors & markers; vertical offsets so points don’t overlap
COLOR_MAP  = {"Precision": "#9ecae1", "F1": "#fc9272", "Recall": "#a1d99b"}  # blue / red / green (light)
MARKER_MAP = {"Precision": "s", "F1": "o", "Recall": "D"}
Y_OFFSETS  = {"Precision": +0.25, "F1": 0.00, "Recall": -0.25}  # Precision up, F1 mid, Recall down

Path(out_dir).mkdir(parents=True, exist_ok=True)

def load_per_model_ci(csv_path: str) -> pd.DataFrame:
    """Load per_model_ci.csv and reshape to long format (no p-values here)."""
    df = pd.read_csv(csv_path)
    # rename Original->OM for display and ordering
    df["trained_on"] = df["trained_on"].replace({"Original": "OM"})
    # build long rows for three metrics
    rows = []
    for _, r in df.iterrows():
        arch = r["architecture"]
        tr   = r["trained_on"]
        rows.append(dict(Architecture=arch, TrainedOn=tr, Metric="Precision",
                         Val=r["precision"], Lo=r["precision_lo"], Hi=r["precision_hi"]))
        rows.append(dict(Architecture=arch, TrainedOn=tr, Metric="F1",
                         Val=r["f1_micro"], Lo=r["f1_lo"], Hi=r["f1_hi"]))
        rows.append(dict(Architecture=arch, TrainedOn=tr, Metric="Recall",
                         Val=r["recall"], Lo=r["recall_lo"], Hi=r["recall_hi"]))
    out = pd.DataFrame(rows)
    # enforce ordering
    out["Architecture"] = pd.Categorical(out["Architecture"], categories=ARCH_ORDER, ordered=True)
    out["TrainedOn"]   = pd.Categorical(out["TrainedOn"],   categories=TRAIN_ORDER, ordered=True)
    out = out.sort_values(["Architecture","TrainedOn","Metric"]).reset_index(drop=True)
    return out

def multiline_label(arch, trained_on):
    # two lines + tiny spacer keeps the three markers visually grouped
    return f"{arch}\n{trained_on}\n"

def plot_forest_on_ax(ax, df_long: pd.DataFrame, title=None, xlim=None, add_legend=False):
    """Per-model forest plot with vertical arch labels + square bracket in left margin (tight symmetric top/bottom)."""
    import numpy as np
    import matplotlib.transforms as mtransforms

    df = df_long.copy()
    df["RowKey"] = df["Architecture"].astype(str) + " · " + df["TrainedOn"].astype(str)
    rows = (
        df.drop_duplicates(subset=["RowKey"])[["Architecture", "TrainedOn", "RowKey"]]
        .sort_values(["Architecture", "TrainedOn"])
    )
    row_keys = rows["RowKey"].tolist()

    # y-base positions (reverse so top is first)
    y_base_map = {rk: i for i, rk in enumerate(row_keys[::-1])}
    df["y_base"] = df["RowKey"].map(y_base_map)

    # ---- plot metrics (Precision↑, F1 mid, Recall↓) ----
    color_map = {"Precision": "#9ecae1", "F1": "#fc9272", "Recall": "#a1d99b"}
    marker_map = {"Precision": "s", "F1": "o", "Recall": "D"}
    offsets    = {"Precision": +0.25, "F1": 0.0, "Recall": -0.25}

    for metric in ["Precision", "F1", "Recall"]:
        sub = df[df["Metric"] == metric]
        y = sub["y_base"].values + offsets[metric]
        x = sub["Val"].values
        xerr = np.vstack([x - sub["Lo"].values, sub["Hi"].values - x])
        ax.errorbar(
            x, y, xerr=xerr,
            fmt=marker_map[metric], color=color_map[metric],
            ecolor=color_map[metric], markersize=5, capsize=0,
            linestyle="None", elinewidth=1.2, zorder=3, label=metric
        )

    # ---- set symmetric y-limits so top/bottom gap = inter-row gap ----
    ybases = sorted(df["y_base"].unique())  # e.g., [0,1,2,...,N-1]
    ax.set_ylim(ybases[0] - 0.5, ybases[-1] + 0.5)  # <<< key line for equal top/bottom spacing

    # ---- thin separators between ticks (behind points) ----
    if xlim is not None:
        ax.set_xlim(xlim)
    x0, x1 = ax.get_xlim()
    for yb in ybases[:-1]:
        ax.hlines(yb + 0.5, x0, x1, colors="#dddddd", linestyles="-", linewidth=0.8, zorder=1)

    # ---- thick black separators between architectures (one tick up) ----
    display_rows = rows.iloc[::-1].reset_index(drop=True)  # top→bottom
    display_arch = display_rows["Architecture"].tolist()
    display_base = [y_base_map[rk] for rk in display_rows["RowKey"]]
    for i in range(len(display_rows) - 1):
        if display_arch[i] != display_arch[i + 1]:
            y_boundary = display_base[i] + 0.5  # one tick up
            ax.hlines(y_boundary, x0, x1, colors="black", linewidth=1.8, zorder=2)

    # ---- y-axis ticks: only labelset names ----
    # ---- y-axis ticks: only labelset names (centered on ticks) ----
    yticks  = [y_base_map[rk] for rk in row_keys[::-1]]
    # REMOVE the trailing '\n' — this is what was pushing labels upward
    ylabels = [f"{tr}" for _, tr, _ in rows.iloc[::-1][["Architecture","TrainedOn","RowKey"]].itertuples(index=False)]
    ax.set_yticks(yticks)
    ax.set_yticklabels(ylabels)

    # Force vertical centering for safety (some backends ignore va via set_yticklabels)
    for lab in ax.get_yticklabels():
        lab.set_va("center")
        # If your labels still look off on your backend, uncomment the next line:
        # lab.set_linespacing(1.0)


    # ---- vertical architecture labels + square bracket in LEFT MARGIN ----
    # axes-x / data-y transform so y aligns to rows; x in axis fraction (margin)
    trans = mtransforms.blended_transform_factory(ax.transAxes, ax.transData)
    # tweak these two to nudge left/right:
    x_bracket = -0.067   # bracket spine (axes fraction; make more negative to go further left)
    x_text    = -0.085   # model text (a bit left of the bracket)
    bracket_tick = 0.015 # tick length on bracket

    i = 0
    total = len(display_rows)
    while i < total:
        arch_i = display_arch[i]
        j = i
        while j < total and display_arch[j] == arch_i:
            j += 1
        bases = [display_base[k] for k in range(i, j)]
        if bases:
            # Span exactly from top Precision (+0.25) to bottom Recall (-0.25)
            y_top    = max(bases) + 0.3   # <<< was +0.6; corrected for symmetry
            y_bottom = min(bases) - 0.25
            y_mid    = 0.5 * (y_top + y_bottom)

            # vertical bracket
            ax.plot([x_bracket, x_bracket], [y_bottom, y_top], color="black", lw=1.2,
                    transform=trans, clip_on=False, zorder=2)
            # top & bottom bracket ticks
            ax.plot([x_bracket, x_bracket + bracket_tick], [y_top, y_top], color="black",
                    lw=1.2, transform=trans, clip_on=False, zorder=2)
            ax.plot([x_bracket, x_bracket + bracket_tick], [y_bottom, y_bottom], color="black",
                    lw=1.2, transform=trans, clip_on=False, zorder=2)

            # vertical model name centered on the block
            ax.text(x_text, y_mid, arch_i, rotation=90, ha="center", va="center",
                    fontsize=10, color="black", weight="bold",
                    transform=trans, clip_on=False, zorder=5)
        i = j

    # ---- labels & legend ----
    ax.set_xlabel("Score (95% CI)")
    if title:
        ax.set_title(title, pad=8)

    if add_legend:
        leg = ax.legend(
            loc="upper center", bbox_to_anchor=(0.5, -0.08),
            ncol=3, frameon=False, handletextpad=0.8
        )
        for t in leg.get_texts():
            t.set_fontsize(9)

    # enough left/bottom room for bracket + legend; adjust if needed
    ax.figure.subplots_adjust(left=0.26, bottom=0.16)
    return ax





# ----- load per-model CIs -----
fixed_ci = load_per_model_ci(os.path.join(fixed_dir, "per_model_ci.csv"))
self_ci  = load_per_model_ci(os.path.join(self_dir,  "per_model_ci.csv"))

# unified x-range for both panels (use [min_lo, max_hi] across both)
xmin = min(fixed_ci["Lo"].min(), self_ci["Lo"].min())
xmax = max(fixed_ci["Hi"].max(), self_ci["Hi"].max())
# pad and clip to [0,1]
xpad = 0.01
xlim = (float(max(0.0, xmin - xpad)), float(min(1.0, xmax + xpad)))

# ----- individual panels (compact legend, minimal bottom whitespace) -----
fig1, ax1 = plt.subplots(figsize=(8, 6.6))
plot_forest_on_ax(
    ax1, fixed_ci,
    title="Per-model micro metrics vs EV reference: Precision / F1 / Recall (95% CI)",
    xlim=xlim, add_legend=True
)
fig1.subplots_adjust(bottom=0.16)
fig1.savefig(os.path.join(out_dir, "per_model_fixed_reference.png"), dpi=300, bbox_inches="tight", pad_inches=0.08)
plt.close(fig1)

fig2, ax2 = plt.subplots(figsize=(8, 6.6))
plot_forest_on_ax(
    ax2, self_ci,
    title="Per-model micro metrics (self-referenced): Precision / F1 / Recall (95% CI)",
    xlim=xlim, add_legend=True
)
fig2.subplots_adjust(bottom=0.16)
fig2.savefig(os.path.join(out_dir, "per_model_self_reference.png"), dpi=300, bbox_inches="tight", pad_inches=0.08)
plt.close(fig2)

# ----- combined side-by-side (legend on right panel only) -----
fig, axes = plt.subplots(1, 2, figsize=(14, 6.6), sharex=True)
plot_forest_on_ax(
    axes[0], fixed_ci,
    title="(A) Fixed-reference (vs EV)",
    xlim=xlim, add_legend=False
)
plot_forest_on_ax(
    axes[1], self_ci,
    title="(B) Self-referenced",
    xlim=xlim, add_legend=True
)
fig.subplots_adjust(wspace=0.25, bottom=0.16)
fig.savefig(os.path.join(out_dir, "per_model_combined_side_by_side.png"), dpi=300, bbox_inches="tight", pad_inches=0.08)
plt.close(fig)

print("Saved:",
      os.path.join(out_dir, "per_model_fixed_reference.png"),
      os.path.join(out_dir, "per_model_self_reference.png"),
      os.path.join(out_dir, "per_model_combined_side_by_side.png"))


Saved: figures/per_model_fixed_reference.png figures/per_model_self_reference.png figures/per_model_combined_side_by_side.png


In [13]:
# Grouped bar charts comparing EV / ER / OM per architecture (with 95% CI error bars)
# Inputs: bootstrap_results/per_model_ci.csv and bootstrap_results_selfref/per_model_ci.csv
# Output: figures/barplot_fixed_reference.png, figures/barplot_self_reference.png, figures/barplot_combined_side_by_side.png

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

fixed_dir = "bootstrap_results"
self_dir  = "bootstrap_results_selfref"
out_dir   = "figures"
Path(out_dir).mkdir(parents=True, exist_ok=True)

ARCH_ORDER = ["LAAT", "CAML", "MultiResCNN", "RNN", "CNN"]
LABELSETS  = ["EV", "ER", "OM"]
METRICS    = ["Precision", "Recall", "F1"]

COLORS = {
    "Precision": "#6baed6",  # blue
    "Recall": "#74c476",     # green
    "F1": "#fb6a4a"          # red
}

def load_per_model_ci(path):
    df = pd.read_csv(path)
    df["trained_on"] = df["trained_on"].replace({"Original": "OM"})
    data = []
    for _, r in df.iterrows():
        data.append(["Precision", r["architecture"], r["trained_on"], r["precision"], r["precision_lo"], r["precision_hi"]])
        data.append(["Recall",    r["architecture"], r["trained_on"], r["recall"],    r["recall_lo"],    r["recall_hi"]])
        data.append(["F1",        r["architecture"], r["trained_on"], r["f1_micro"],  r["f1_lo"],        r["f1_hi"]])
    df_long = pd.DataFrame(data, columns=["Metric","Architecture","LabelSet","Val","Lo","Hi"])
    df_long["Architecture"] = pd.Categorical(df_long["Architecture"], ARCH_ORDER, ordered=True)
    df_long["LabelSet"] = pd.Categorical(df_long["LabelSet"], LABELSETS, ordered=True)
    return df_long

def plot_grouped_bars(df_long, title, save_path):
    fig, axes = plt.subplots(1, len(ARCH_ORDER), figsize=(15, 5), sharey=True)
    for i, arch in enumerate(ARCH_ORDER):
        ax = axes[i]
        sub = df_long[df_long["Architecture"] == arch]
        bar_width = 0.25
        x = np.arange(len(LABELSETS))
        offsets = [-bar_width, 0, bar_width]  # precision, recall, f1 positions

        for j, metric in enumerate(METRICS):
            sm = sub[sub["Metric"] == metric].sort_values("LabelSet")
            y = sm["Val"].values
            yerr = [y - sm["Lo"].values, sm["Hi"].values - y]
            ax.bar(x + offsets[j], y, width=bar_width, color=COLORS[metric],
                   label=metric if i == 0 else "", yerr=yerr, capsize=3, edgecolor="black", linewidth=0.5)

        ax.set_xticks(x)
        ax.set_xticklabels(LABELSETS)
        ax.set_title(arch)
        ax.set_ylim(0, 1)
        ax.grid(axis="y", linestyle="--", linewidth=0.5, alpha=0.6)
        if i == 0:
            ax.set_ylabel("Score (95% CI)")

    fig.suptitle(title, fontsize=13, y=0.98)
    fig.legend(METRICS, loc="lower center", ncol=3, bbox_to_anchor=(0.5, -0.02))
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(save_path, dpi=300, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)
    print(f"Saved {save_path}")

# Load data
fixed_df = load_per_model_ci(os.path.join(fixed_dir, "per_model_ci.csv"))
self_df  = load_per_model_ci(os.path.join(self_dir,  "per_model_ci.csv"))

# Plot both
plot_grouped_bars(fixed_df,
                  "Per-model micro Precision / Recall / F1 (Fixed-reference vs EV)",
                  f"{out_dir}/barplot_fixed_reference.png")

plot_grouped_bars(self_df,
                  "Per-model micro Precision / Recall / F1 (Self-referenced)",
                  f"{out_dir}/barplot_self_reference.png")

# Combined (side by side)
fig, axs = plt.subplots(2, 1, figsize=(15, 9), sharex=True)
for i, (df, title) in enumerate([
    (fixed_df, "(A) Fixed-reference vs EV"),
    (self_df, "(B) Self-referenced")
]):
    for j, arch in enumerate(ARCH_ORDER):
        ax = plt.subplot(2, len(ARCH_ORDER), i*len(ARCH_ORDER)+j+1)
        sub = df[df["Architecture"] == arch]
        bar_width = 0.25
        x = np.arange(len(LABELSETS))
        offsets = [-bar_width, 0, bar_width]
        for k, metric in enumerate(METRICS):
            sm = sub[sub["Metric"] == metric].sort_values("LabelSet")
            y = sm["Val"].values
            yerr = [y - sm["Lo"].values, sm["Hi"].values - y]
            ax.bar(x + offsets[k], y, width=bar_width, color=COLORS[metric],
                   yerr=yerr, capsize=3, edgecolor="black", linewidth=0.5)
        ax.set_xticks(x)
        ax.set_xticklabels(LABELSETS)
        ax.set_title(arch)
        ax.set_ylim(0, 1)
        ax.grid(axis="y", linestyle="--", linewidth=0.5, alpha=0.6)
        if j == 0:
            ax.set_ylabel("Score (95% CI)")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.figtext(0.5, 0.01, "Label Set", ha="center", fontsize=11)
plt.suptitle("Micro Precision / Recall / F1 across Label Sets", fontsize=13)
plt.savefig(f"{out_dir}/barplot_combined_side_by_side.png", dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.close()


Saved figures/barplot_fixed_reference.png
Saved figures/barplot_self_reference.png
